### Example 1.

In [ ]:
%load_ext Cython

In [6]:
%%cython
def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

In [7]:
%timeit geo_prog_cython(0.99, int(10**6))

10 loops, best of 3: 41.7 ms per loop


### Example 2. 
#### Cython with numpy arrays

In [9]:
%%cython
import numpy as np

def qm_cython_first_pass(double x0, int n):
    cdef int t
    x = np.zeros(n+1, float)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

In [10]:
%timeit qm_cython_first_pass(0.1, int(10**5))

10 loops, best of 3: 44.4 ms per loop


In [11]:
%%cython
import numpy as np
from numpy cimport float_t

def qm_cython(double x0, int n):
    cdef int t
    x_np_array = np.zeros(n+1, dtype=float)
    cdef float_t [:] x = x_np_array
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

In [12]:
%timeit qm_cython(0.1, int(10**5))

1000 loops, best of 3: 479 µs per loop


### Problem 1.

In [13]:
import matplotlib.pyplot as plt
import numpy as np
from numba import jit

p, q = 0.1, 0.2

In [14]:
def compute_series(n):
    x = np.empty(n, dtype=int)
    x[0] = 1  # Start in state 1
    U = np.random.uniform(0, 1, size=n)
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return x

In [16]:
compute_series_numba = jit(compute_series)

In [17]:
%%cython
import numpy as np
from numpy cimport int_t, float_t

def compute_series_cy(int n):
    # == Create NumPy arrays first == #
    x_np = np.empty(n, dtype=int)
    U_np = np.random.uniform(0, 1, size=n)
    # == Now create memoryviews of the arrays == #
    cdef int_t [:] x = x_np
    cdef float_t [:] U = U_np
    # == Other variable declarations == #
    cdef float p = 0.1
    cdef float q = 0.2
    cdef int t
    # == Main loop == #
    x[0] = 1
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return np.asarray(x)

In [19]:
n = 1000
%timeit compute_series(n)

1000 loops, best of 3: 660 µs per loop


In [22]:
%timeit compute_series_numba(n)

The slowest run took 9.31 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 19.6 µs per loop


In [24]:
%timeit compute_series_cy(n)

10000 loops, best of 3: 29.1 µs per loop
